In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import eigh as sp_eigh
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
def LDA(X, y, n_components):
    n_samples, n_features = X.shape
    cl, cls_counts = np.unique(y)
    priors = cls_counts / n_samples
    X_cls_mean = np.array([X[y == cls].mean(axis=0) for cls in cl])
    between_cls_deviation = X_cls_mean - X.mean(axis=0)
    within_cls_deviation = X - X_cls_mean[y]
    Sb = priors * between_cls_deviation.T @ between_cls_deviation
    Sw = within_cls_deviation.T @ within_cls_deviation / n_samples
    evals, evecs = sp_eigh(Sb, Sw)
    dvecs = evecs[:, np.argsort(evals)[::-1]]
    w = X_cls_mean @ dvecs @ dvecs.T
    bias = np.log(priors) - 0.5 * np.diag(X_cls_mean @ w.T)
    if n_components is None:
        n_components = min(cl.size - 1, n_features)
    return X @ dvecs[:, : n_components]

LDA vs PCA
simularity:
1)банальное, оба метода предназначены для уменьшения размерности данных, что упрощает анализ, визуализацию и снижает вычислительную сложность
2)Для обоих методов важно предварительно стандартизировать данные, чтобы признаки имели одинаковую шкалу
3)Оба метода проецируют исходные данные на новое подпространство меньшей размерности
4) Оба метода находят новые оси, при этом максимизируя какой-то параметр, но эти параметры разные для каждого метода
Differences:
1) PCA находит оси, которые максимизируют дисперсию данных.А LDA находит оси, которые максимизируют различие между классами и минимизируют разброс внутри классов.
2)PCA применяется для предварительной обработки, кластеризации и визуализации. Может использоваться для данных без меток. То есть обучение без учителя.
LDA же используется для улучшения разделимости классов в задачах классификации. Требует наличия меток классов. То есть обучение с учителем
